In [5]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests
import datetime
from progressbar import *
from IPython.display import clear_output
from urllib3.util import url

In [6]:
sections = ["https://www.dunyahalleri.com/category/haftanin-ozeti/page/",
            "https://www.dunyahalleri.com/category/genel-gundem/page/",
            "https://www.dunyahalleri.com/category/teknoloji-bilim/page/",
            "https://www.dunyahalleri.com/category/internet-girisimler/page/",
            "https://www.dunyahalleri.com/category/tasarim-inovasyon/page/",
            "https://www.dunyahalleri.com/category/kultur-sanat/page/"]

urls = []

In [7]:
for section in sections:
    # Kategorinin içerisinde sırayla 100 sayfa gezineceğiz.
    for i in range(1, 30):
        try:
            # Öncelikle URL'imizi oluşturuyoruz. Örneğin;
            # https://www.dunyahalleri.com/category/kultur-sanat/page/25
            newurl = section + str(i)
            print(newurl)

            # Url'nin içerisindeki bütün html dosyasını indiriyoruz.
            html = requests.get(newurl).text
            soup = bs(html, "xml")

            # Yukarıdaki şemadada görüldüğü gibi bütün makaleler bu  element'in içerisinde yer alıyor.
            # Bizde bütün makaleleri buradan tags adında bir değişkene topluyoruz.
            tags = soup.findAll("div", class_="row row-eq-height herald-posts")[0]

            # Sırayla bütün makalelere girip, href'in içerisindeki linki urls adlı listemize append ediyoruz.
            for a in tags.find_all('a', href=True):
                urls.append((section.split("/")[4], a['href']))
        except IndexError:
            break

urldata = pd.DataFrame(urls)
urldata.columns = ["Kategori", "Link"]
urldata.head()

urldata = urldata.drop_duplicates()
urldata.to_csv('urldata.csv')

https://www.dunyahalleri.com/category/haftanin-ozeti/page/1
https://www.dunyahalleri.com/category/haftanin-ozeti/page/2
https://www.dunyahalleri.com/category/haftanin-ozeti/page/3
https://www.dunyahalleri.com/category/haftanin-ozeti/page/4
https://www.dunyahalleri.com/category/haftanin-ozeti/page/5
https://www.dunyahalleri.com/category/haftanin-ozeti/page/6
https://www.dunyahalleri.com/category/haftanin-ozeti/page/7
https://www.dunyahalleri.com/category/haftanin-ozeti/page/8
https://www.dunyahalleri.com/category/haftanin-ozeti/page/9
https://www.dunyahalleri.com/category/haftanin-ozeti/page/10
https://www.dunyahalleri.com/category/haftanin-ozeti/page/11
https://www.dunyahalleri.com/category/haftanin-ozeti/page/12
https://www.dunyahalleri.com/category/haftanin-ozeti/page/13
https://www.dunyahalleri.com/category/haftanin-ozeti/page/14
https://www.dunyahalleri.com/category/haftanin-ozeti/page/15
https://www.dunyahalleri.com/category/haftanin-ozeti/page/16
https://www.dunyahalleri.com/cate

In [16]:
def GetData(url):
    try:
        if 'http' in url:
            #Url içerisindeki html'i indiriyoruz.
            html = requests.get(url).text
            soup = bs(html, "lxml")
            body_text = soup.find_all('div', class_="herald-entry-content")[0].find_all('p')

            #Body_text adındaki metni tek bir string üzerinde topluyoruz.
            body_text_big = ""
            for i in body_text:
                body_text_big = body_text_big + i.text

            #Başlığı ve zamanı'da element isimlerinden bu şekilde seçip, metinlerini alıyoruz.
            header = soup.find("h1", class_="entry-title h1").text
            if soup.find("span", class_="updated"):
                timestamp = soup.find("span", class_="updated").text
            else:
                timestamp = '01/01/2016'

            #Özetin bulunduğu element'in metin kısmını alıyoruz.
            summarized = "this is a summary"
            return ((url, header, body_text_big, summarized, timestamp))
        else:
            return ("Boş Data")

    #Link boş ise verilen hata üzerine Boş Data mesajını dönüyor.
    except IndexError:
        return ("Boş Data")

    #Eğer link haftalık özet ise özet kısmı olmadığından oraya haftalık özet yazıp, sonuçlar o şekilde dönüyor.
    except AttributeError:
        return ((url, header, body_text_big, "Haftalık Özet", timestamp))

bigdata = []
k = 0

for i in urldata.Link:
    if(k < 100):
        clear_output(wait=True)
        print(k)
        temp = GetData(i)
        bigdata.append(temp)
        k = k + 1
    else:
        break
#Verileri DataFrame olarak kaydetme
bigdatax = pd.DataFrame(bigdata)
bigdatax.drop([5, 6, 7], axis=1, inplace=True)
bigdatax.drop(bigdatax[bigdatax[0] == "B"].index, axis=0, inplace=True)
bigdatax.columns = ["Link", "Başlık", "Body_text", "Summarized_Text", "TimeStamp"]
bigdatax = bigdatax.loc[bigdatax.Link.drop_duplicates().index]
bigdatax.index = range(0, len(bigdatax))
bigdatax.head()

99


,Link,Başlık,Body_text,Summarized_Text,TimeStamp
0,https://www.dunyahalleri.com/hoscakalin/,Hoşçakalın,Dünya Halleri yayın hayatına 2 Kasım 2014 tari...,this is a summary,06/08/2019
1,https://www.dunyahalleri.com/haftanin-spor-oze...,Haftanın Spor Özeti: 59,"Haftanın Spor Özeti, Dünya Halleri özetlerinin...",this is a summary,05/08/2019
2,https://www.dunyahalleri.com/haftanin-ozeti-249/,Haftanın Özeti: 249,Dünyada bir yanda açlık çeken insanlar diğer y...,this is a summary,04/08/2019
3,https://www.dunyahalleri.com/dizel-araclari-el...,Dizel araçları elektrikliye dönüştüren girişim,Haber ÖzetiTam SürümFransız girişimi Transitio...,this is a summary,03/08/2019
4,https://www.dunyahalleri.com/ed-sheeranin-turn...,Ed Sheeran’ın turnesi U2’nun rekorunu kıracak,Haber ÖzetiTam SürümEd Sheeran Divide turnesin...,this is a summary,03/08/2019


In [27]:
with pd.ExcelWriter('DunyaHalleri.xlsx',  options={'strings_to_urls': False}) as writer:
    bigdatax.to_excel(writer, sheet_name='Sheet_name_1')

C:\Users\win10\AppData\Local\Temp\ipykernel_2420\715847890.py:1: FutureWarning: Use of **kwargs is deprecated, use engine_kwargs instead.
  with pd.ExcelWriter('DunyaHalleri.xlsx',  options={'strings_to_urls': False}) as writer:


In [29]:
# Haber içeriği excel'inin içeri alınması
data = pd.read_excel('DunyaHalleri.xlsx')
data.head()

,Unnamed: 0,Link,Başlık,Body_text,Summarized_Text,TimeStamp
0,0,https://www.dunyahalleri.com/hoscakalin/,Hoşçakalın,Dünya Halleri yayın hayatına 2 Kasım 2014 tari...,this is a summary,06/08/2019
1,1,https://www.dunyahalleri.com/haftanin-spor-oze...,Haftanın Spor Özeti: 59,"Haftanın Spor Özeti, Dünya Halleri özetlerinin...",this is a summary,05/08/2019
2,2,https://www.dunyahalleri.com/haftanin-ozeti-249/,Haftanın Özeti: 249,Dünyada bir yanda açlık çeken insanlar diğer y...,this is a summary,04/08/2019
3,3,https://www.dunyahalleri.com/dizel-araclari-el...,Dizel araçları elektrikliye dönüştüren girişim,Haber ÖzetiTam SürümFransız girişimi Transitio...,this is a summary,03/08/2019
4,4,https://www.dunyahalleri.com/ed-sheeranin-turn...,Ed Sheeran’ın turnesi U2’nun rekorunu kıracak,Haber ÖzetiTam SürümEd Sheeran Divide turnesin...,this is a summary,03/08/2019


In [34]:
#Görselleştirme
from IPython.display import display,HTML,IFrame
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.cm as cm
import base64
%matplotlib inline
import plotly.express as px
sns.set()
#NLP ve Modelleme
from collections import Counter
import gensim
from gensim.models import Word2Vec
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from scattertext import CorpusFromPandas, produce_scattertext_explorer
import scattertext as st
from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from pprint import pprint
from scipy.stats import rankdata, hmean, norm
import spacy

In [35]:
#Veriler link üzerinden eşleşecekler, Eşleşmeyen linkler için kategori boş olarak gelecek.
data = pd.merge(data, urldata, on='Link', how='left')
data.head()

,Unnamed: 0,Link,Başlık,Body_text,Summarized_Text,TimeStamp,Kategori_x,Kategori_y
0,0,https://www.dunyahalleri.com/hoscakalin/,Hoşçakalın,Dünya Halleri yayın hayatına 2 Kasım 2014 tari...,this is a summary,06/08/2019,haftanin-ozeti,haftanin-ozeti
1,1,https://www.dunyahalleri.com/haftanin-spor-oze...,Haftanın Spor Özeti: 59,"Haftanın Spor Özeti, Dünya Halleri özetlerinin...",this is a summary,05/08/2019,haftanin-ozeti,haftanin-ozeti
2,2,https://www.dunyahalleri.com/haftanin-ozeti-249/,Haftanın Özeti: 249,Dünyada bir yanda açlık çeken insanlar diğer y...,this is a summary,04/08/2019,haftanin-ozeti,haftanin-ozeti
3,3,https://www.dunyahalleri.com/dizel-araclari-el...,Dizel araçları elektrikliye dönüştüren girişim,Haber ÖzetiTam SürümFransız girişimi Transitio...,this is a summary,03/08/2019,haftanin-ozeti,haftanin-ozeti
4,4,https://www.dunyahalleri.com/ed-sheeranin-turn...,Ed Sheeran’ın turnesi U2’nun rekorunu kıracak,Haber ÖzetiTam SürümEd Sheeran Divide turnesin...,this is a summary,03/08/2019,haftanin-ozeti,haftanin-ozeti


In [36]:
def preprocess(ReviewText):
    #Verideki <br> taglarını kaldır.
    ReviewText = ReviewText.str.replace("(<br/>)", "")
    ReviewText = ReviewText.str.replace('(<a).*(>).*(</a>)', '')
    ReviewText = ReviewText.str.replace('(&amp)', '')
    ReviewText = ReviewText.str.replace('(&gt)', '')
    ReviewText = ReviewText.str.replace('(&lt)', '')
    ReviewText = ReviewText.str.replace('(\xa0)', ' ')
    #Verideki Linkleri Kaldır.
    ReviewText = ReviewText.str.replace(r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', ' ')
    return ReviewText
data['Body_text'] = preprocess(data['Body_text'])

C:\Users\win10\AppData\Local\Temp\ipykernel_2420\3038642604.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  ReviewText = ReviewText.str.replace("(<br/>)", "")
C:\Users\win10\AppData\Local\Temp\ipykernel_2420\3038642604.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  ReviewText = ReviewText.str.replace('(<a).*(>).*(</a>)', '')
C:\Users\win10\AppData\Local\Temp\ipykernel_2420\3038642604.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  ReviewText = ReviewText.str.replace('(&amp)', '')
C:\Users\win10\AppData\Local\Temp\ipykernel_2420\3038642604.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  ReviewText = ReviewText.str.replace('(&gt)', '')
C:\Users\win10\AppData\Local\Temp\ipykernel_2420\3038642604.py:7: FutureWarning: The default value of regex will change from True to False in

In [37]:
data['yogunlasma'] = data['Body_text'].map(lambda text: TextBlob(text).sentiment.polarity)
data['harf_uzunlugu'] = data['Body_text'].astype(str).apply(len)
data['kelime_sayisi'] = data['Body_text'].apply(lambda x: len(str(x).split()))

In [39]:
fig = px.histogram(data, x="kelime_sayisi", nbins=50, title='Kelime Sayısı')
fig.show()

In [40]:
fig = px.histogram(data, x="harf_uzunlugu", nbins=200, title='Metin Uzunluğu')
fig.show()

In [43]:
def get_top_n_words(corpus, n=None):
    vec = CountVectorizer().fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_words(data['Body_text'], 25)
df1 = pd.DataFrame(common_words, columns = ['kelime' , 'geçiş frekansı'])
fig = px.bar(df1, x='kelime', y='geçiş frekansı',
             hover_data=['kelime', 'geçiş frekansı'], color='geçiş frekansı',
             title='Stopwordsleri kaldırmadan en çok geçen 25 kelime',
             height=400)
fig.show()

In [48]:
import nltk
nltk.download('stopwords')
stop = stopwords.words('Turkish')
with open('turkce-stop-words.txt', encoding='utf-8') as file:
    stw = file.read()
stw = stw.split()
stw = [s.lower() for s in stw]
stop += stw
def get_top_n_words(corpus, n=None):
    vec = CountVectorizer(stop_words = stop).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_words(data['Body_text'], 25)
df2 = pd.DataFrame(common_words, columns = ['kelime' , 'geçiş frekansı'])
fig = px.bar(df2, x='kelime', y='geçiş frekansı',
             hover_data=['kelime', 'geçiş frekansı'], color='geçiş frekansı',
             title='Stopwordsleri kaldırdıktan sonra en çok geçen 25 kelime',
             height=400)
fig.show()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\win10\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [49]:
def get_top_n_bigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(2, 2), stop_words=stop).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_bigram(data['Body_text'], 25)
df3 = pd.DataFrame(common_words, columns = ['kelime' , 'geçiş frekansı'])
fig = px.bar(df3, x='kelime', y='geçiş frekansı',
             hover_data=['kelime', 'geçiş frekansı'], color='geçiş frekansı',
             title='Stopwordsleri kaldırdıktan sonra en çok yan yana gelen ikili kelimeler',
             height=400)
fig.show()

In [54]:
#Öncelikle text_data adında yeni bir dataframe oluşturup, bunda sadece haber metinleri ve kategorilerini tutuyorum.
text_data = data[['Body_text','Kategori_x']]
text_data.head()

,Body_text,Kategori_x
0,Dünya Halleri yayın hayatına 2 Kasım 2014 tari...,haftanin-ozeti
1,"Haftanın Spor Özeti, Dünya Halleri özetlerinin...",haftanin-ozeti
2,Dünyada bir yanda açlık çeken insanlar diğer y...,haftanin-ozeti
3,Haber ÖzetiTam SürümFransız girişimi Transitio...,haftanin-ozeti
4,Haber ÖzetiTam SürümEd Sheeran Divide turnesin...,haftanin-ozeti
